In [1]:
%reload_ext autoreload
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from pipetools import pipe, utils, where
from fastcore.basics import typed
import requests
from fastcore.foundation import L
import pandas as pd
from typing import List
from pathlib import Path
import re
from datetime import datetime, date

In [21]:
import sys
sys.path.append("../../")

In [22]:
from src.run import run_fbref
from src.fbref.utilities import format_age, get_proxy_format, to_float

In [33]:
stats = run_fbref(debug=True)

2021-01-09 15:48:32.705 | INFO     | src.run:fetch_proxies:86 - Fetching proxies...
2021-01-09 15:48:35.234 | DEBUG    | src.run:fetch_proxies:102 - [FETCH] Proxies are:
['http://45.145.56.68:7634:cocrxyts-dest:5f5p1h28ekyr', 'http://45.72.55.228:7265:cocrxyts-dest:5f5p1h28ekyr', 'http://45.86.15.48:8595:cocrxyts-dest:5f5p1h28ekyr', 'http://185.95.157.172:6193:cocrxyts-dest:5f5p1h28ekyr', 'http://45.130.255.211:7233:cocrxyts-dest:5f5p1h28ekyr']
2021-01-09 15:48:35.236 | INFO     | src.run:run_fbref:29 - [TEAM:LIVERPOOL] Fetching data from Fbref
2021-01-09 15:48:38.711 | INFO     | src.run:run_fbref:35 - [TEAM:LIVERPOOL] Example Shooting stats: ShootingStats(name='Andrew Robertson', profile_url='https://fbref.com/en/players/2e4f5f03/Andrew-Robertson', nationality=['sco', 'SCO'], position='DF', age='26 years and 304 days', minutes_90s=17.0, goals=1, shots_total=7, shots_on_target=2, shots_on_target_pct=28.6, shots_total_per90=0.41, goals_per_shot=0.14, goals_per_shot_on_target=0.5, shots

In [41]:
stats_available = ["shooting_stats", "passing_stats", "extra_passing_stats", "gca_stats",
                   "defensive_action_stats", "possession_stats", "playing_time_stats", "misc_stats"]

In [42]:
stats["team"]

('liverpool', 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats')

In [47]:
# for statsa in stats_available:
#     print([stat._asdict() for stat in stats[statsa]])

## FPL Data

In [2]:
fpl_request = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")

In [3]:
all_fpl_data = fpl_request.content

In [4]:
all_fpl_data = json.loads(all_fpl_data)

In [5]:
all_fpl_data.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [6]:
all_fpl_data["total_players"]

7818404

In [7]:
fpl_players = L(all_fpl_data["elements"])

In [8]:
fpl_teams = L(all_fpl_data["teams"])

In [9]:
fpl_teams[0]

{'code': 3,
 'draw': 0,
 'form': None,
 'id': 1,
 'loss': 0,
 'name': 'Arsenal',
 'played': 0,
 'points': 0,
 'position': 0,
 'short_name': 'ARS',
 'strength': 4,
 'team_division': None,
 'unavailable': False,
 'win': 0,
 'strength_overall_home': 1190,
 'strength_overall_away': 1210,
 'strength_attack_home': 1170,
 'strength_attack_away': 1210,
 'strength_defence_home': 1190,
 'strength_defence_away': 1200,
 'pulse_id': 1}

In [10]:
@typed
def extract_team_details(team: dict) -> dict:
    return {str(team["code"]): {"name": team["name"].lower().replace(" ", "_"), "slug": team["short_name"]}}

In [11]:
fpl_team_mapper = {}
[fpl_team_mapper.update(extract_team_details(team)) for team in fpl_teams]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [12]:
fpl_team_mapper

{'3': {'name': 'arsenal', 'slug': 'ARS'},
 '7': {'name': 'aston_villa', 'slug': 'AVL'},
 '36': {'name': 'brighton', 'slug': 'BHA'},
 '90': {'name': 'burnley', 'slug': 'BUR'},
 '8': {'name': 'chelsea', 'slug': 'CHE'},
 '31': {'name': 'crystal_palace', 'slug': 'CRY'},
 '11': {'name': 'everton', 'slug': 'EVE'},
 '54': {'name': 'fulham', 'slug': 'FUL'},
 '13': {'name': 'leicester', 'slug': 'LEI'},
 '2': {'name': 'leeds', 'slug': 'LEE'},
 '14': {'name': 'liverpool', 'slug': 'LIV'},
 '43': {'name': 'man_city', 'slug': 'MCI'},
 '1': {'name': 'man_utd', 'slug': 'MUN'},
 '4': {'name': 'newcastle', 'slug': 'NEW'},
 '49': {'name': 'sheffield_utd', 'slug': 'SHU'},
 '20': {'name': 'southampton', 'slug': 'SOU'},
 '6': {'name': 'spurs', 'slug': 'TOT'},
 '35': {'name': 'west_brom', 'slug': 'WBA'},
 '21': {'name': 'west_ham', 'slug': 'WHU'},
 '39': {'name': 'wolves', 'slug': 'WOL'}}

In [13]:
all_fpl_data["events"][0]

{'id': 1,
 'name': 'Gameweek 1',
 'deadline_time': '2020-09-12T10:00:00Z',
 'average_entry_score': 50,
 'finished': True,
 'data_checked': True,
 'highest_scoring_entry': 4761681,
 'deadline_time_epoch': 1599904800,
 'deadline_time_game_offset': 0,
 'highest_score': 142,
 'is_previous': False,
 'is_current': False,
 'is_next': False,
 'chip_plays': [{'chip_name': 'bboost', 'num_played': 112843},
  {'chip_name': '3xc', 'num_played': 225426}],
 'most_selected': 259,
 'most_transferred_in': 12,
 'top_element': 254,
 'top_element_info': {'id': 254, 'points': 20},
 'transfers_made': 0,
 'most_captained': 4,
 'most_vice_captained': 4}

In [14]:
all_fpl_data["phases"][3]

{'id': 4, 'name': 'November', 'start_event': 8, 'stop_event': 10}

In [15]:
category_types = {}
[category_types.update({str(player_cat["id"]): player_cat["singular_name_short"]}) for player_cat in all_fpl_data["element_types"]]

[None, None, None, None]

In [16]:
category_types

{'1': 'GKP', '2': 'DEF', '3': 'MID', '4': 'FWD'}

In [17]:
all_fpl_data["element_stats"]

[{'label': 'Minutes played', 'name': 'minutes'},
 {'label': 'Goals scored', 'name': 'goals_scored'},
 {'label': 'Assists', 'name': 'assists'},
 {'label': 'Clean sheets', 'name': 'clean_sheets'},
 {'label': 'Goals conceded', 'name': 'goals_conceded'},
 {'label': 'Own goals', 'name': 'own_goals'},
 {'label': 'Penalties saved', 'name': 'penalties_saved'},
 {'label': 'Penalties missed', 'name': 'penalties_missed'},
 {'label': 'Yellow cards', 'name': 'yellow_cards'},
 {'label': 'Red cards', 'name': 'red_cards'},
 {'label': 'Saves', 'name': 'saves'},
 {'label': 'Bonus', 'name': 'bonus'},
 {'label': 'Bonus Points System', 'name': 'bps'},
 {'label': 'Influence', 'name': 'influence'},
 {'label': 'Creativity', 'name': 'creativity'},
 {'label': 'Threat', 'name': 'threat'},
 {'label': 'ICT Index', 'name': 'ict_index'}]

In [18]:
@typed
def extract_player_fpl_stats(player: dict) -> dict:
    return {
        "player_id": player["id"],
        "name": f"{player['first_name']} {player['second_name']}",
        "display_slug": player["web_name"],
        "team_id": player["team"],
        "team": fpl_team_mapper.get(str(player["team_code"])).get("name"),
        "team_slug": fpl_team_mapper.get(str(player["team_code"])).get("slug"),
        "position": category_types.get(str(player["element_type"])),
        "points_per_game": to_float(player["points_per_game"]),
        "form": to_float(player["form"]),
        "news": player["news"],
        "news_added_at": player["news_added"],
        "status": player["status"],
        "total_points": player["total_points"],
        "value_form": to_float(player["value_form"]),
        "value_season": to_float(player["value_season"]),
        "selected_by_percent": to_float(player["selected_by_percent"]),
        "points_per_game": to_float(player["points_per_game"]),
        "influence_rank_overall": player["influence_rank"],
        "creativity_rank_overall": player["creativity_rank"],
        "threat_rank_overall": player["threat_rank"],
        "influence_rank_by_position": player["influence_rank_type"],
        "creativity_rank_by_position": player["creativity_rank_type"],
        "threat_rank_by_position:": player["threat_rank_type"],
        "ict_index": player["ict_index_rank"],
        "ict_index_by_position": player["ict_index_rank_type"],
        "dreamteam_count": player["dreamteam_count"],
        "transfers_in": player["transfers_in"],
        "transfers_out": player["transfers_out"],
        "chance_of_playing_next_round": player["chance_of_playing_next_round"],
        "chance_of_playing_this_round": player["chance_of_playing_this_round"],
        "cost_change_event": player["cost_change_event"],
        "cost_change_event_fall": player["cost_change_event_fall"],
        "cost_change_start": player["cost_change_start"],
        "cost_change_start_fall": player["cost_change_start_fall"],
        "ep_next": player["ep_next"],
        "ep_this": player["ep_this"],
        "now_cost": player["now_cost"],
        "transfers_in": player["transfers_in"],
        "transfers_out": player["transfers_out"],
        "transfers_in_event": player["transfers_in_event"],
        "transfers_out_event": player["transfers_out_event"],
        "goals_scored": player["goals_scored"],
        "assists": player["assists"],
        "clean_sheets": player["clean_sheets"],
        "goals_conceded": player["goals_conceded"],
        "penalties_saved": player["penalties_saved"],
        "penalties_missed": player["penalties_missed"],
        "saves": player["saves"],
        "bonus": player["bonus"],
        "bps": player["bps"],
        "corners_and_indirect_freekicks_order": player["corners_and_indirect_freekicks_order"],
        "corners_and_indirect_freekicks_text": player["corners_and_indirect_freekicks_text"],
        "direct_freekicks_order": player["direct_freekicks_order"],
        "direct_freekicks_text": player["direct_freekicks_text"],
        "penalties_order": player["penalties_order"],
        "penalties_text": player["penalties_text"],
    }

In [23]:
%time fpl_player_stats = L([extract_player_fpl_stats(player) for player in fpl_players])

CPU times: user 31.2 ms, sys: 0 ns, total: 31.2 ms
Wall time: 30.3 ms


In [24]:
fpl_player_stats_df = pd.DataFrame.from_dict(fpl_player_stats)

In [25]:
FPL_DATA = Path("../../data/fpl-data")

In [26]:
date.today()

datetime.date(2021, 1, 9)

In [27]:
t_now = re.sub(r'\D', '', str(date.today()))

#### Check gameweek aggregated stats

In [28]:
all_fpl_data["events"][1]

{'id': 2,
 'name': 'Gameweek 2',
 'deadline_time': '2020-09-19T10:00:00Z',
 'average_entry_score': 59,
 'finished': True,
 'data_checked': True,
 'highest_scoring_entry': 6234344,
 'deadline_time_epoch': 1600509600,
 'deadline_time_game_offset': 0,
 'highest_score': 165,
 'is_previous': False,
 'is_current': False,
 'is_next': False,
 'chip_plays': [{'chip_name': 'bboost', 'num_played': 94615},
  {'chip_name': 'freehit', 'num_played': 111968},
  {'chip_name': 'wildcard', 'num_played': 494000},
  {'chip_name': '3xc', 'num_played': 221133}],
 'most_selected': 259,
 'most_transferred_in': 302,
 'top_element': 390,
 'top_element_info': {'id': 390, 'points': 24},
 'transfers_made': 14637421,
 'most_captained': 4,
 'most_vice_captained': 254}

In [29]:
live_gameweek = 18

In [30]:
FPL_DATA.mkdir(parents=True, exist_ok=True)
fpl_player_stats_df.to_csv(FPL_DATA / f"{t_now}_GW{live_gameweek}_player_stats.csv")

### Basic EDA

In [31]:
fpl_player_stats_df.head()

,player_id,name,display_slug,team_id,team,team_slug,position,points_per_game,form,news,...,penalties_missed,saves,bonus,bps,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,1,Mesut Özil,Özil,1,arsenal,ARS,MID,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,...,0,0,0,0,NaN,,NaN,,NaN,
1,2,Sokratis Papastathopoulos,Sokratis,1,arsenal,ARS,DEF,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,...,0,0,0,0,NaN,,NaN,,NaN,
2,3,David Luiz Moreira Marinho,David Luiz,1,arsenal,ARS,DEF,1.1,0.3,,...,0,0,0,70,NaN,,4.0,,NaN,
3,4,Pierre-Emerick Aubameyang,Aubameyang,1,arsenal,ARS,MID,3.5,2.7,,...,0,0,4,154,NaN,,NaN,,1.0,
4,5,Cédric Soares,Cédric,1,arsenal,ARS,DEF,1.0,0.2,,...,0,0,0,3,NaN,,NaN,,NaN,


In [32]:
fpl_player_stats_df.groupby(["team_slug", "position"])["player_id"].count().reset_index().rename(
    columns={'player_id': 'count'})

,team_slug,position,count
0,ARS,DEF,13
1,ARS,FWD,2
2,ARS,GKP,3
3,ARS,MID,15
4,AVL,DEF,10
...,...,...,...
75,WHU,MID,11
76,WOL,DEF,14
77,WOL,FWD,4
78,WOL,GKP,4


### Reading on statistics

- [ICT Index](https://www.premierleague.com/news/65567)